In [2]:
import gzip
import os

import pandas as pd
import numpy as np

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
root_dir = '/content/drive/MyDrive/LaboDatos_1C2022/tp_final/data/Imdb/'
files = os.listdir(root_dir)

imdb_dict = {}
for file in files:
  dataset_name = file.split('.')
  file_dir = os.path.join(root_dir, file)
#  with gzip.open(os.path.join(root_dir, file), 'rb') as f:        
  if 'name' in dataset_name:
    imdb_dict[dataset_name[0]] = pd.read_csv(file_dir, sep='\t', na_values='\\N',
                                             low_memory=False, compression='gzip')
  
  else:  
    imdb_dict[dataset_name[1]] = pd.read_csv(file_dir, sep='\t', na_values='\\N',
                                             low_memory=False, compression='gzip')


In [ ]:
df_titles = imdb_dict['akas']
df_basics = imdb_dict['basics']
df_ratings = imdb_dict['ratings']
df_crew = imdb_dict['crew']
df_principals = imdb_dict['principals']
df_names = imdb_dict['names']

del imdb_dict

KeyError: ignored

### Title

In [ ]:
df_titles.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


### Title basics

In [ ]:
df_basics['titleType'].unique()

array(['short', 'movie', 'tvEpisode', 'tvSeries', 'tvShort', 'tvMovie',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [ ]:
df_basics[df_basics['tconst']=='tt0113497'] # tiene generos distintos

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
110942,tt0113497,movie,Jumanji,Jumanji,0.0,1995.0,NaN,104.0,"Adventure,Comedy,Family"


In [ ]:
# me quedo solo con las de tipo Movie
df_movies = df_basics[df_basics['titleType']=='movie']

In [ ]:
df_movies.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
498,tt0000502,movie,Bohemios,Bohemios,0.0,1905.0,NaN,100.0,NaN
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,NaN,70.0,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0.0,1907.0,NaN,90.0,Drama
610,tt0000615,movie,Robbery Under Arms,Robbery Under Arms,0.0,1907.0,NaN,NaN,Drama
625,tt0000630,movie,Hamlet,Amleto,0.0,1908.0,NaN,NaN,Drama


In [ ]:
df_movies['tconst'].apply(lambda x: x[2:]).astype(int) # REVISAR BIEN ESTO

498            502
570            574
587            591
610            615
625            630
            ...   
8961626    9916622
8961653    9916680
8961665    9916706
8961676    9916730
8961686    9916754
Name: tconst, Length: 611473, dtype: int32

In [ ]:
df_movies['endYear'].unique()

array([nan])

In [ ]:
df_movies['isAdult'].unique()

array([0., 1.])

In [ ]:
df_movies['isAdult'].value_counts()

0.0    602060
1.0      9413
Name: isAdult, dtype: int64

In [ ]:
df_movies.drop(columns=['endYear', 'titleType'], inplace=True)

C:\Users\DAIANA_NUBI\anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [ ]:
df_movies['genres'] = df_movies['genres'].apply(lambda x: x.split(',') if type(x)==str else x)

<ipython-input-24-a81987407057>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movies['genres'] = df_movies['genres'].apply(lambda x: x.split(',') if type(x)==str else x)


In [ ]:
df_movies.head()

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
498,tt0000502,Bohemios,Bohemios,0.0,1905.0,100.0,NaN
570,tt0000574,The Story of the Kelly Gang,The Story of the Kelly Gang,0.0,1906.0,70.0,"[Action, Adventure, Biography]"
587,tt0000591,The Prodigal Son,L'enfant prodigue,0.0,1907.0,90.0,[Drama]
610,tt0000615,Robbery Under Arms,Robbery Under Arms,0.0,1907.0,NaN,[Drama]
625,tt0000630,Hamlet,Amleto,0.0,1908.0,NaN,[Drama]


### Ratings

In [ ]:
df_ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1883
1,tt0000002,5.9,250
2,tt0000003,6.5,1667
3,tt0000004,5.8,163
4,tt0000005,6.2,2490


In [ ]:
df_ratings.shape[0]

1249576

In [ ]:
df_final = df_movies.merge(df_ratings, on='tconst', how='left') #611473

### Title crew

In [ ]:
df_crew_filter = df_crew[df_crew['tconst'].isin(df_movies['tconst'].unique())]

In [ ]:
df_crew_filter.head() # puede haber más de un director y más de un escritor

,tconst,directors,writers
498,tt0000502,nm0063413,"nm0063413,nm0657268,nm0675388"
570,tt0000574,nm0846879,nm0846879
587,tt0000591,nm0141150,nm0141150
610,tt0000615,nm0533958,"nm0092809,nm0533958"
625,tt0000630,nm0143333,nm0000636


In [ ]:
df_crew_filter['directors'] = df_crew_filter['directors'].apply(lambda x: x.split(',') if type(x)==str else x)
df_crew_filter['writers'] = df_crew_filter['writers'].apply(lambda x: x.split(',') if type(x)==str else x)

<ipython-input-32-19e9867e9645>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_crew_filter['directors'] = df_crew_filter['directors'].apply(lambda x: x.split(',') if type(x)==str else x)
<ipython-input-32-19e9867e9645>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_crew_filter['writers'] = df_crew_filter['writers'].apply(lambda x: x.split(',') if type(x)==str else x)


In [ ]:
df_directors = df_crew_filter.assign(director=df_crew_filter['directors']).explode('director').drop(columns=['writers', 'directors'])
df_writers = df_crew_filter.assign(writer=df_crew_filter['writers']).explode('writer').drop(columns=['directors', 'writers'])

In [ ]:
# Me voy a quedar solo con estos ids
ids_directors = df_directors['director'].unique()
ids_writers = df_writers['writer'].unique()

In [ ]:
print(len(ids_directors))
print(len(ids_writers))

229788
320698


### Names

In [ ]:
df_names.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899.0,1987.0,"soundtrack,actor,miscellaneous","tt0053137,tt0072308,tt0050419,tt0031983"
1,nm0000002,Lauren Bacall,1924.0,2014.0,"actress,soundtrack","tt0037382,tt0117057,tt0071877,tt0038355"
2,nm0000003,Brigitte Bardot,1934.0,NaN,"actress,soundtrack,music_department","tt0049189,tt0056404,tt0054452,tt0057345"
3,nm0000004,John Belushi,1949.0,1982.0,"actor,soundtrack,writer","tt0077975,tt0072562,tt0078723,tt0080455"
4,nm0000005,Ingmar Bergman,1918.0,2007.0,"writer,director,actor","tt0050986,tt0050976,tt0083922,tt0060827"


In [ ]:
df_directors_names = df_names[df_names['nconst'].isin(ids_directors)]
df_writers_names = df_names[df_names['nconst'].isin(ids_writers)]

In [ ]:
df_directors_names.shape[0] # hay 29 que no estaban en df_names

229759

In [ ]:
df_writers_names.shape[0] # hay 37 que no estaban en df_names

320661

In [ ]:
df_directors_names = df_directors_names[['nconst', 'primaryName']]
df_writers_names = df_writers_names[['nconst', 'primaryName']]

In [ ]:
df_directors_names.rename(columns={'nconst': 'director'}, inplace=True)
df_writers_names.rename(columns={'nconst': 'writer'}, inplace=True)

In [ ]:
df_directors = df_directors.merge(df_directors_names, on='director', how='left')

In [ ]:
df_writers = df_writers.merge(df_writers_names, on='writer', how='left')

In [ ]:
df_directors['tconst'].nunique()

611473

In [ ]:
df_directors = df_directors.groupby('tconst').agg({'director': 'unique', 'primaryName': 'unique'}).reset_index()

In [ ]:
df_directors.head()

,tconst,director,primaryName
0,tt0000502,[nm0063413],[Ricardo de Baños]
1,tt0000574,[nm0846879],[Charles Tait]
2,tt0000591,[nm0141150],[Michel Carré]
3,tt0000615,[nm0533958],[Charles MacMahon]
4,tt0000630,[nm0143333],[Mario Caserini]


In [ ]:
df_directors.shape[0]

611473

In [ ]:
df_writers = df_writers.groupby('tconst').agg({'writer': 'unique', 'primaryName': 'unique'}).reset_index()

In [ ]:
df_writers.head()

,tconst,writer,primaryName
0,tt0000502,"[nm0063413, nm0657268, nm0675388]","[Ricardo de Baños, Miguel de Palacios, Guiller..."
1,tt0000574,[nm0846879],[Charles Tait]
2,tt0000591,[nm0141150],[Michel Carré]
3,tt0000615,"[nm0092809, nm0533958]","[Rolf Boldrewood, Charles MacMahon]"
4,tt0000630,[nm0000636],[William Shakespeare]


In [ ]:
df_writers.shape[0]

611473

### Merge final

In [ ]:
df_final.shape[0]

611473

In [ ]:
df_final = df_final.merge(df_directors, on='tconst')

In [ ]:
df_final = df_final.merge(df_writers, on='tconst')

In [ ]:
df_final = df_final.drop(columns=['director', 'writer']).rename(columns={'primaryName_x': 'directors', 'primaryName_y': 'writers'})

In [ ]:
df_final.shape[0]

611473

In [ ]:
# Verificación
df_final[df_final['tconst']=='tt0113497'] 

,tconst,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres,averageRating,numVotes,directors,writers
78661,tt0113497,Jumanji,Jumanji,0.0,1995.0,104.0,"[Adventure, Comedy, Family]",7.0,337453.0,[Joe Johnston],"[Jonathan Hensleigh, Greg Taylor, Jim Strain, ..."


In [ ]:
df_final.to_csv("data_completa_imdb.csv", index=False)